In [1]:
from pandas.api.types import is_numeric_dtype
import sys
sys.path.append('..')
from src.data_utils import collect_results
import os
dir_name = '../results/Llama-3.1-8B-Instruct'
model_name = dir_name.split('/')[-1]

df = collect_results(dir_name)
df = df.sort_values('micro_f1', ascending=False)
df.round(3)
for col in df.columns:
    if is_numeric_dtype(df[col]):
        df[col] = df[col].fillna(0)
        if (df[col] == 0).all():
            df = df.drop(col, axis=1)
# col_set = set(df.columns)
# col_set.remove('seed')
df = df.drop(['model_name',], axis = 1)
df['epoch'] = df['epoch'].apply(lambda x: x + 1)
df_group = (
    df.groupby(['tags', 'n_icl'])
      .agg(['mean', 'std'])  # compute both mean and std
      .round(3)
)
# df_group.columns = ['_'.join(col) for col in df_group.columns]  # flatten MultiIndex
df_group = df_group.reset_index()
# df_group.insert(loc=0, column='model', value=model_name)
df_group = df_group[df_group.columns[:14]]
latex_filename = '../paper/results_micro.tex'
os.makedirs(os.path.dirname(latex_filename), exist_ok=1)
df_group.to_latex(latex_filename, label='results_micro', caption=f'Micro-averaged results for {model_name}.', float_format="%.3f", index=False, escape=True)
df_group

tags n_icl seed        epoch          ft      micro_f1_dev        micro_f1  \
             mean    std  mean    std mean  std         mean    std     mean   
0    n     0  2.0  1.581   3.0  0.000  1.0  0.0        0.366  0.027    0.377   
1    n     5  2.0  1.581   2.0  0.707  1.0  0.0        0.389  0.027    0.392   
2    n    10  2.0  1.581   1.8  0.447  1.0  0.0        0.406  0.027    0.389   
3    n    15  2.0  1.581   1.0  0.000  1.0  0.0        0.277  0.022    0.279   
4    n    20  2.0  1.581   1.0  0.000  1.0  0.0        0.279  0.077    0.269   
5    y     0  2.0  1.581   3.0  0.000  1.0  0.0        0.338  0.023    0.336   
6    y     5  2.0  1.581   2.2  0.837  1.0  0.0        0.218  0.014    0.204   
7    y    10  2.0  1.581   2.6  0.894  1.0  0.0        0.213  0.023    0.209   
8    y    15  2.0  1.581   2.4  0.894  1.0  0.0        0.217  0.022    0.217   
9    y    20  2.0  1.581   2.6  0.894  1.0  0.0        0.210  0.029    0.203   

         micro_p         
     std    mean    std  
0  0.043   0.423  0.052  
1  0.041   0.412  0.038  
2  0.068   0.413  0.094  
3  0.034   0.283  0.027  
4  0.042   0.265  0.034  
5  0.053   0.385  0.061  
6  0.039   0.163  0.031  
7  0.040   0.170  0.036  
8  0.046   0.172  0.046  
9  0.040   0.163  0.031

In [ ]:
from pandas.api.types import is_numeric_dtype
import sys
sys.path.append('..')
from src.data_utils import collect_results
import os
dir_name = '../results/Llama-3.3-70B-Instruct'
model_name = dir_name.split('/')[-1]

df = collect_results(dir_name)
df = df.sort_values('micro_f1', ascending=False)
df.round(3)
print(df)
for col in df.columns:
    if is_numeric_dtype(df[col]):
        df[col] = df[col].fillna(0)
        if (df[col] == 0).all():
            df = df.drop(col, axis=1)

df = df.drop(['model_name',], axis = 1)
df['epoch'] = df['epoch'].apply(lambda x: x+1)
df_group = (
    df.drop(columns=['seed'])  # marginalize over seed
      .groupby(['tags', 'n_icl', 'ft'])
      .agg(['mean', 'std'])  # compute both mean and std
      .round(3)
)
# df_group.columns = ['_'.join(col) for col in df_group.columns]  # flatten MultiIndex
df_group = df_group.reset_index()
# df_group.insert(loc=0, column='model', value=model_name)
df_group = df_group[df_group.columns[:12]]
latex_filename = '../paper/results_micro.tex'
os.makedirs(os.path.dirname(latex_filename), exist_ok=1)
df_group.to_latex(latex_filename, label='results_micro', caption=f'Micro-averaged results for {model_name}.', float_format="%.3f", index=False, escape=True)
df_group

               model_name  seed tags  n_icl  epoch  ft  micro_f1_dev  \
9  Llama-3.3-70B-Instruct     3    n      5      1   1      0.203905   
7  Llama-3.3-70B-Instruct     3    n      5      0   0      0.000000   
2  Llama-3.3-70B-Instruct     2    n      5      0   1      0.241636   
4  Llama-3.3-70B-Instruct     2    n      5      0   0      0.000000   
0  Llama-3.3-70B-Instruct     0    n      5      0   0      0.000000   
5  Llama-3.3-70B-Instruct     0    n      5      2   1      0.234375   
6  Llama-3.3-70B-Instruct     4    n      5      0   1      0.241681   
1  Llama-3.3-70B-Instruct     4    n      5      0   0      0.000000   
8  Llama-3.3-70B-Instruct     1    n      5      0   0      0.000000   
3  Llama-3.3-70B-Instruct     1    n      5      0   1      0.229299   

   micro_f1   micro_p   micro_r  ...  NGT  GINF  XVNE  PUNC  XVBE  NMA   NC  \
9  0.241935  0.234375  0.250000  ...  NaN   NaN   NaN   0.0   0.0  0.0  0.0   
7  0.241803  0.237903  0.245833  ...  NaN   NaN  

tags n_icl ft epoch        micro_f1_dev        micro_f1        micro_p  \
                 mean    std         mean    std     mean    std    mean   
0    n     5  0   1.0  0.000         0.00  0.000    0.206  0.036   0.202   
1    n     5  1   1.6  0.894         0.23  0.016    0.209  0.034   0.206   

         micro_r  
     std    mean  
0  0.035   0.210  
1  0.035   0.212

In [3]:
import sys
import re
import pandas as pd
sys.path.append('..')
from src.data_utils import convert_files, extract_tags
df = convert_files('../data')
df['tags'] = df.apply(lambda x: [el[0][0] for el in extract_tags(x['text_an'])], axis = 1)

def flatten_deep(lst):
    return [item for sublist in lst 
            for item in (flatten_deep(sublist) if isinstance(sublist, list) else [sublist])]
tags_list = flatten_deep(df['tags'])
tags_list = pd.Series(tags_list)
counts = tags_list.value_counts()
print(tags_list)
print(counts)
sum((counts.values < 10).astype(int))
# sum((tags_list > 9).int())

0       C
1       C
2       D
3       D
4       D
       ..
2670    F
2671    D
2672    F
2673    X
2674    D
Length: 2675, dtype: object
D    950
G    622
F    319
L    307
W    163
X    114
Q    102
Z     54
C     44
Name: count, dtype: int64


np.int64(0)